In [104]:
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [105]:
# Store csv created in part one into a DataFrame
cities_weather = pd.read_csv("city_weather.csv")
cities_weather = cities_weather.rename(columns={"Unnamed: 0": "City_ID"})
cities_weather.head()

,City_ID,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
0,0,Baracoa,20.3467,-74.4958,78.66,76.0,17.0,11.01,CU,1.617054e+09
1,1,Yellowknife,62.4560,-114.3525,6.01,47.0,20.0,13.80,CA,1.617053e+09
2,2,Hobart,-42.8794,147.3294,52.00,87.0,0.0,4.61,AU,1.617054e+09
3,3,Hermanus,-34.4187,19.2345,66.00,61.0,32.0,3.29,ZA,1.617054e+09
4,4,Rocha,-34.4833,-54.3333,65.62,90.0,70.0,12.46,UY,1.617054e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [118]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_weather[['latitude', 'longitude']].astype(float)
humidity = cities_weather['humidity'].astype(float)

fig_layout = {
        'width': '1000px',
        'height': '420px',
        'padding': '3px',
        'border': '1px solid black'
}

fig = gmaps.figure(layout = fig_layout, center=(46.0, -5.0), zoom_level=1.5)


# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_weather['humidity']),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [119]:
# Narrow down cities that fit criteria of:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_weather = cities_weather.loc[(cities_weather['max temp'] > 70.00) & (cities_weather['max temp'] < 90.00) & 
                                   (cities_weather['wind speed'] < 10) & (cities_weather['cloudiness'] == 0)]

ideal_weather.head()

,City_ID,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
11,11,Beïnamar,8.6698,15.3813,82.67,16.0,0.0,5.10,TD,1.617054e+09
175,197,Caravelas,-17.7125,-39.2481,79.03,71.0,0.0,7.87,BR,1.617053e+09
192,216,Saint-Philippe,-21.3585,55.7679,77.00,78.0,0.0,6.91,RE,1.617053e+09
202,227,Conde,-7.2597,-34.9075,80.60,89.0,0.0,4.61,BR,1.617054e+09
214,239,General Roca,-39.0333,-67.5833,84.20,13.0,0.0,6.91,AR,1.617053e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [120]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather[['city_name', 'country', 'latitude', 'longitude']]
# hotel_df = hotel_df.rename(columns={"city_name": "City", "country": "Country", "latitude": "Lat", "longitude": "Lng"})
# hotel_df['Coordinates'] = list(zip(hotel_df['latitude'], hotel_df['longitude']))
hotel_df['Hotel Name'] = ""
hotel_df['Coordinates'] = list(zip(hotel_df['latitude'], hotel_df['longitude']))
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(['index'],axis=1)
hotel_df.head()

<ipython-input-120-fd71c46a5243>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""
<ipython-input-120-fd71c46a5243>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Coordinates'] = list(zip(hotel_df['latitude'], hotel_df['longitude']))


,city_name,country,latitude,longitude,Hotel Name,Coordinates
0,Beïnamar,TD,8.6698,15.3813,,"(8.6698, 15.3813)"
1,Caravelas,BR,-17.7125,-39.2481,,"(-17.7125, -39.2481)"
2,Saint-Philippe,RE,-21.3585,55.7679,,"(-21.3585, 55.7679)"
3,Conde,BR,-7.2597,-34.9075,,"(-7.2597, -34.9075)"
4,General Roca,AR,-39.0333,-67.5833,,"(-39.0333, -67.5833)"


In [121]:
#convert coordinates columns to a string and remove parathesis
hotel_df['Coordinates'] = hotel_df['Coordinates'].astype(str)
hotel_df['Coordinates'] = hotel_df['Coordinates'].str.replace("(", "")
hotel_df['Coordinates'] = hotel_df['Coordinates'].str.replace(")", "")

In [122]:
# Set parameters to search for a hotel
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    params["location"] = hotel_df['Coordinates'][index]

    # make request and print url and convert to json
    response = requests.get(url, params=params).json()
#     pprint(response)
    
    #Grab the first hotel from the results and store the name in the hotel_df dataframe
    #get first hotel names and which cities had no hotels within radius
    try:
        print(response['results'][0]['name'])
        hotel_df['Hotel Name'][index] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df['Hotel Name'][index] = ""

Missing field/result... skipping.


<ipython-input-122-c8eac73e0d74>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][index] = ""


Pousada dos Navegantes


<ipython-input-122-c8eac73e0d74>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][index] = response['results'][0]['name']


Chambres d'hôte "La Trinité"
Pousada Beija Flor
Hotel y Casino del Río General Roca
Bungalows Guesthouse
شهدان. ابوالحجر
HAMDAN PLAZA HOTEL SALALAH
Missing field/result... skipping.
Lindsey Hôtel
Huarpes Hotel
Quest Palmerston - Serviced Apartments and Accommodation
Diego De Almagro Coyhaique
Hotel Provincial
AXIM BEACH RESORT AND SPA
پلکه بزرگ
Paihuen
"Plantation Bed and Breakfast


In [123]:
hotel_df

,city_name,country,latitude,longitude,Hotel Name,Coordinates
0,Beïnamar,TD,8.6698,15.3813,,"8.6698, 15.3813"
1,Caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes,"-17.7125, -39.2481"
2,Saint-Philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité""","-21.3585, 55.7679"
3,Conde,BR,-7.2597,-34.9075,Pousada Beija Flor,"-7.2597, -34.9075"
4,General Roca,AR,-39.0333,-67.5833,Hotel y Casino del Río General Roca,"-39.0333, -67.5833"
5,Tahoua,NE,14.8888,5.2692,Bungalows Guesthouse,"14.8888, 5.2692"
6,Jizan,SA,17.3333,42.6667,شهدان. ابوالحجر,"17.3333, 42.6667"
7,Salalah,OM,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH,"17.0151, 54.0924"
8,Leer,SS,8.3018,30.1418,,"8.3018, 30.1418"
9,Saint-Pierre,RE,-21.3393,55.4781,Lindsey Hôtel,"-21.3393, 55.4781"


In [124]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [125]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# template = gmaps.marker_layer(hotel_info)

# Display figure
fig.add_layer(markers)
# fig.add_layer(template)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px', width='1000px'))